In [0]:
folder_name=['customer','location','trips','payment','driver','vehicle']

### Function For Incemental Loading

In [0]:
def data_ingestion(folder_name):
        # Schema Read
        df=spark.read.format('csv')\
             .option("header","true")\
             .option("inferSchema","true")\
             .load(f"/Volumes/Project2/default/source/{folder_name}/")
        schema = df.schema

        # Incemental Data Ingestion
        df=spark.readStream.format('csv')\
                    .schema(schema)\
                    .option("header","true")\
                    .load(f'/Volumes/Project2/default/source/{folder_name}/')
        df.writeStream.format('delta')\
                    .outputMode('append')\
                    .option("checkpointLocation", f"/Volumes/Project2/bronze/bronze_volume/checkpoint/{folder_name}/")\
                    .option('mergeSchema','true')\
                    .trigger(once=True)\
                    .toTable(f'Project2.bronze.{folder_name}')

### Mutilthreading For Concurrent Processing

In [0]:
from concurrent.futures import ThreadPoolExecutor

with ThreadPoolExecutor(max_workers=len(folder_name)) as executor:
    executor.map(data_ingestion, folder_name)